In [22]:
# define rooms and items

freedom = {
    'name': 'freedom',
    'type': 'room',
}

# Sala 6 --> Calle
outside = {
  'name': 'outside',
  'type': 'room',
}
dog = {
    'name': 'dog',
    'type': 'door',
}

# Sala 1 --> CELDA
cell = {
    'name':'cell',
    'type': 'room',
}
bunk_bed = {
  'name':'bunk bed',
  'type':'furniture',
}
toilet = {
  'name':'toilet',
  'type':'furniture',
}
washbain = {
  'name':'washbain',
  'type':'furniture',
}
cell_door = {
  'name':'cell door',
  'type':'door',
}
lock_pick = {
  'name':'lock pick',
  'type': 'key',
  'target': cell_door,
}
# Sala 2 --> Pasillo
corridor = {
    'name': 'corridor',
    'type': 'room',
}
dining_room_door = {
  'name':'dining room door',
  'type':'door',
}
infirmary_door = {
  'name':'infirmary door',
  'type':'door',
}
camera_room_door = {
  'name':'camera room door',
  'type':'door',
}
guard ={
  'name':'guard',
  'type':'furniture',
}
card_key={
  'name':'card key',
  'type':'key',
  'target': camera_room_door,
}
# Sala 3 --> Comedor
dining_room ={
    'name':'dining room',
    'type': 'room',
}
cutlery_drawer= {
    'name': 'cutlery drawer',
    'type': 'furniture',
}
fork ={
  'name':'fork',
  'type':'key',
  'target': infirmary_door,
}
kitchen ={
  'name':'kitchen',
  'type':'furniture',
}
dog_food = {
  'name':'food',
  'type': 'key',
  'target': dog,
}
window = {
  'name': 'window',
  'type': 'furniture',
}

# Sala 4 --> Sala Cámaras
camera_room = {
    'name': 'camera room',
    'type': 'room'
}
computer = {
    'name': 'computer',
    'type': 'furniture',
}
recordings = {
    'name': 'recordings',
    'type': 'key',
}
drawer = {
    'name': 'drawer',
    'type': 'furniture',
}
paper_with_code = {
    'name': 'paper with code',
    'type': 'key',
    'target': dining_room_door,
    'codes': ['138', '428', '311'],
    'correct_code': '311',
}

# Sala 5 --> Enfermería
infirmary = {
    'name': 'infirmary',
    'type': 'room',
}
window_exit = {
    'name': 'window exit',
    'type': 'door',
}
pill = {
    'name': 'pills',
    'type': 'key',
}
table = {
    'name': 'table',
    'type': 'furniture',
}
scalpel = {
    'name': 'scalpel',
    'type': 'key',
    'target': window_exit,
}

all_rooms = [cell, corridor, dining_room, camera_room, infirmary , outside, freedom]

all_doors = [dining_room_door, cell_door, infirmary_door, window_exit, dog, camera_room_door ]

# define which items/rooms are related

object_relations = {
    'cell': [bunk_bed, toilet, washbain, cell_door],
    'toilet': [lock_pick],
    'cell door': [corridor,cell],
    'corridor': [dining_room_door, cell_door, camera_room_door, guard],
    'guard':[card_key],
    'dining room':[dining_room_door, kitchen, window, cutlery_drawer, infirmary_door],
    'dining room door': [dining_room, corridor],
    'kitchen': [dog_food],
    'cutlery drawer': [fork],
    'camera room':[camera_room_door, computer, drawer],
    'camera room door':[camera_room,corridor],
    'computer': [recordings],
    'drawer':[paper_with_code],
    'infirmary': [infirmary_door, window_exit, pill, table],
    'table':[scalpel],
    'infirmary door': [infirmary, dining_room],
    'window exit': [infirmary, outside],
    'outside': [window_exit, dog],
    'dog':[outside, freedom],
    'dog_food':[dog],
    'freedom':[dog],
}

In [23]:
INIT_GAME_STATE = {
    "current_room": cell,
    "keys_collected": [],
    "target_room": freedom
}

In [24]:
def start_game():
    print("You open your eyes in the dark, cold cell of Fox River State Penitentiary.\n"
          "Your pulse races as memories flood back: framed for a crime you didn’t commit.\n"
          "You’ve been planning this escape for months. Everything depends on breaking out of here tonight.\n"
          "You glance around, scanning your cell. The guards will make their rounds soon.\n"
          "Time’s ticking. You’ve got to act fast. There’s no going back.\n"
          "Every second counts.\n")
    play_room(game_state["current_room"])

def play_room(room):
    game_state["current_room"] = room
    if room["name"] == "outside" and "dog" in [item["name"] for item in object_relations[room["name"]]]:
        print("You encounter a fierce dog blocking your way to freedom.")

        if any(item["name"] == "food" for item in game_state["keys_collected"]):
            print("You have dog food! You give the food to the dog, and it lets you pass.")
            return print("Congrats! You have escaped the prison!")
        else:
            print("You don't have any food for the dog. The dog growls menacingly, forcing you to retreat.")
            game_state["keys_collected"] = []
            print("The dog chased you back to the cell. You have lost all your items!")
            play_room(cell)

    else:
        print(f"You’re now in {room['name']}. What will you do next?")
        intended_action = input("Type '1' to explore/look around, or '2' to examine an item: ").strip()

        if intended_action == '1':
            explore_room(room)
            play_room(room)
        elif intended_action == '2':
            items = [i["name"] for i in object_relations[room["name"]]]
            item_to_examine = input("What would you like to examine? ").strip().lower()

            if item_to_examine in [item.lower() for item in items]:
                examine_item(item_to_examine)
            else:
                print("Invalid input. Please choose a valid item from the room.")
                play_room(room)
        else:
            print("You can’t waste time. Type '1' to explore or '2' to examine before it’s too late.")
            play_room(room)

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You check " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if item_name == "dining room door":
                    code_correct=False
                    while not code_correct:
                        try:
                            code = input("Codes to open the door you found on the paper are: 138, 428, 311. Write one of those: ")
                            if not code.isdigit() or len(code) != 3:
                                raise ValueError("The code must be a 3-digit number.")

                            if code == "311":
                                code_correct = True
                                output += "You have entered the correct code. Unlocked door."
                                next_room = get_next_room_of_door(item, current_room)
                                break
                            else:
                                print('You have entered the wrong code. Locked door.')
                        except ValueError as e:
                            print(f"Error: {e} Please enter a valid 3-digit code.")
                elif(have_key):
                    output += "You unlock it with the key you stole. The way forward is clear."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "The door’s locked tight. You need a key—and fast."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "Jackpot. You find a " + item_found["name"] + "."
                else:
                    output += "You find nothing. Time is slipping away."

            print(output)

            break
    if(output is None):
        print("Nothing here. Focus. Every second counts.")

    if(next_room and input("Do you want to move forward? Type 'yes' or 'no':'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)


In [25]:
game_state = INIT_GAME_STATE.copy()

start_game()

You open your eyes in the dark, cold cell of Fox River State Penitentiary.
Your pulse races as memories flood back: framed for a crime you didn’t commit.
You’ve been planning this escape for months. Everything depends on breaking out of here tonight.
You glance around, scanning your cell. The guards will make their rounds soon.
Time’s ticking. You’ve got to act fast. There’s no going back.
Every second counts.

You’re now in cell. What will you do next?
Type '1' to explore/look around, or '2' to examine an item: 1
You explore the room. This is cell. You find bunk bed, toilet, washbain, cell door
You’re now in cell. What will you do next?
Type '1' to explore/look around, or '2' to examine an item: 2
What would you like to examine? toilet
You check toilet. Jackpot. You find a lock pick.
You’re now in cell. What will you do next?
Type '1' to explore/look around, or '2' to examine an item: 2
What would you like to examine? cell door
You check cell door. You unlock it with the key you stole